<a href="https://colab.research.google.com/github/guebin/DL2024/blob/main/posts/13wk-2.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 1. 강의영상 

In [1]:
#{{<video https://youtu.be/playlist?list=PLQqh36zP38-zoOHd7w3N5q9Jc5P34Ux8X&si=MdJTHM3a27MCAssp >}}

# 2. Imports 

In [2]:
import numpy as np

# 3. 강화학습 Intro 

`-` 강화학습(대충설명): 어떠한 "(게임)환경"이 있을때 거기서 "뭘 할지"를 학습하는 과업

`-` 딥마인드: breakout $\to$ 알파고 

- <https://www.youtube.com/watch?v=TmPfTpjtdgg>

`-` 강화학습 미래? (이거 잘하면 먹고 살 수 있을까?) 

# 4. Game1: `Bandit` 게임

## A. 게임설명 및 원시코드

`-` 문제설명: 두 개의 버튼이 있다. `버튼0`을 누르면 1의 보상을, `버튼1`을 누르면 10의 보상을 준다고 가정 

`-` 처음에 어떤 행동을 해야 하는가?

- 처음에는 아는게 없음
- 일단 "아무거나" 눌러보자. 

`-` 버튼을 아무거나 누르는 코드를 작성해보자. 

> `action_space` 와 `action` 이라는 용어를 기억할 것 

`-` 버튼을 누른 행위에 따른 보상을 구현하자. 

> `reward`라는 용어를 기억할 것 

`-` 아무버튼이나 10번정도 눌러보면서 데이터를 쌓아보자. 

`-` 깨달았음: `버튼0`을 누르면 1점을 받고, `버튼1`을 누르면 10점을 받는 "환경(environment)"이구나? $\to$ `버튼1`을 누르는 "동작(=action)"을 해야하는 상황이구나? 

- 여기에서 $\to$의 과정을 체계화 시킨 학문이 강화학습 

> `environment`라는 용어를 기억할 것 

- 게임 클리어

`-` 강화학습: 환경(environment)을 이해 $\to$ 에이전트(agent)가 행동(action)을 결정 

> `agent`라는 용어를 기억할 것 

***위의 과정이 잘 되었다는 의미로 사용하는 문장들*** 

- 강화학습이 성공적으로 잘 되었다. 
- 에이전트가 환경의 과제를 완료했다. 
- 에이전트가 환경에서 성공적으로 학습했다. 
- 에이전트가 올바른 행동을 학습했다. 
- 게임 클리어 (비공식) 

`-` 게임이 클리어 되었다는 것을 의미하는 지표를 정하고 싶다. 

- 첫 생각: `버튼1`을 누르는 순간 게임클리어로 보면 되지 않나?
- 두번째 생각: 아니지? 우연히 누를수도 있잖아?
- 게임클리어조건: (1) 20번은 그냥 진행 (2) 최근 20번의 보상의 평균이 95점 이상이면 게임이 클리어 되었다고 생각하자.^[`버튼1`을 눌러야 하는건 맞지만 몇번의 실수는 눈감아 주자는 의미]

`-` 원시코드1: 환경을 이해하지 못한 에이전트 -- 게임을 클리어할 수 없다. 

`-` 원시코드2: 환경을 깨달은 에이전트 -- 게임클리어 

## B. 수정1: `Env` 구현 

`-` `Bandit` 클래스 선언 + `.step()` 구현 

## C. 수정2: `Agent` 구현 (인간지능)

`-` Agent 클래스 설계

- 액션을 하고, 본인의 행동과 환경에서 받은 reward를 기억 
- `.act()`함수와 `.save_experience()`함수 구현 

--- 대충 아래와 같은 느낌으로 코드가 돌아가요 ---

**시점0**: init 

In [137]:
env = Bandit()
agent = Agent() 

In [138]:
agent.action, agent.reward

(None, None)

**시점1**: agent >> env 

In [139]:
agent.act()

In [140]:
agent.action, agent.reward

(1, None)

In [141]:
env.agent_action = agent.action

**시점2**: agent << env 

In [142]:
agent.reward = env.step(env.agent_action)

In [143]:
agent.action, agent.reward, env.agent_action

(1, 10, 1)

In [144]:
agent.actions,agent.rewards

([], [])

In [145]:
agent.save_experience()

In [146]:
agent.actions,agent.rewards

([1], [10])

-- 전체코드 -- 

## D. 수정3: `Agent` 구현 (인공지능)

`-` 지금까지 풀이의 한계

- 사실 강화학습은 "환경을 이해 $\to$ 행동을 결정" 의 과정에서 "$\to$"의 과정을 수식화 한 것이다.
- 그런데 지금까지 했던 코드는 환경(environment)를 이해하는 순간 에이전트(agent)가 최적의 행동(action)^[`버튼1`을 누른다]을 **"직관적으로"** 결정하였으므로 기계가 스스로 학습을 했다고 볼 수 없다. 

`-` 에이전트가 데이터를 보고 스스로 학습할 수 있도록 설계 -- 부제: `agent.learn()`을 설계하자. 

1. 데이터를 모아서 `q_table` 를 만든다. `q_table`은 아래와 같은 내용을 포함한다. 

|행동|보상(추정값)|
|:--:|:--:|
|버튼0 ($=a_0$)|1 ($=q_0$)|
|버튼1 ($=a_1$)|100 ($=q_1$)|

2. `q_table`을 바탕으로 적절한 정책(=`policy`)을 설정한다. 

- 이 예제에서는 버튼0과 버튼1을 각각 $\big(\frac{q_0}{q_0+q_1},\frac{q_1}{q_0+q_1}\big)$ 의 확률로 선택하는 "정책"을 이용하면 충분할 듯 

> 여기에서 `q_table`, `policy`라는 용어를 기억하세요.

`-` `q_table`을 계산하는 코드 예시

In [ ]:
agent.actions = [0, 1, 1,  0, 1,   0, 0] 
agent.rewards = [1, 9, 10, 1, 9.5, 1, 1.2] 
actions = np.array(agent.actions)
rewards = np.array(agent.rewards)

`-` 최종코드정리 